In [11]:
import numpy as np
import pickle
import tensorflow as tf

from torch.autograd import Variable

In [35]:
class PMF(tf.keras.Model):
    def __init__(self, n_users, n_items, n_dim):
        super(PMF, self).__init__()
        ## initializing attributes from parameters        
        self.w_u_i_init = tf.keras.initializers.RandomUniform(minval=-.01, maxval=.01, seed=1)
        
        ## initializing Embedding layers
        self.user_embedding = tf.keras.layers.Embedding(n_users,
                                                        n_dim,
                                                        embeddings_initializer='uniform',
                                                        embeddings_regularizer=tf.keras.regularizers.L2(0.1))
        
        self.item_embedding = tf.keras.layers.Embedding(n_items,
                                                        n_dim,
                                                        embeddings_initializer='uniform',
                                                        embeddings_regularizer=tf.keras.regularizers.L2(0.1))
        
        self.ub = tf.keras.layers.Embedding(n_users, 1, embeddings_initializer=self.w_u_i_init, embeddings_regularizer=tf.keras.regularizers.L2(0.1))
        self.ib = tf.keras.layers.Embedding(n_items, 1, embeddings_initializer=self.w_u_i_init, embeddings_regularizer=tf.keras.regularizers.L2(0.1))
        
    def call(self, user_index, item_index):
        user_h1 = self.user_embedding(user_index)
        item_h1 = self.item_embedding(item_index)
        
        r_h = tf.math.reduce_sum(user_h1 * item_h1) + tf.squeeze(self.ub(user_index)) + tf.squeeze(self.ib(item_index))
        return r_h

In [40]:
print('===== TRAINING PMF =====')
## initializing hyperparameters
batch_size = 64
epoches = 150
seed = 1
weight_decay = 0.1
emb_dim = 100
ratio = 0.8
lr = 1e-3
momentum = 0.9

## loading dataset
users = pickle.load(open('dataset/user_id_to_num_mov.pkl', 'rb'))
items = pickle.load(open('dataset/movie_id_to_num_mov.pkl', 'rb'))
data = np.load('dataset/data.npy')

print("===== Dataset has been loaded =====")

## As the paper implemented, normalizing the rating
## that will act as reward
data[:,2] = 0.5 * (data[:, 2] - 3)

## Shuffle data
rndg = np.random.default_rng(seed=42)
rndg.shuffle(data)
 
## Splitting data data
train_data = data[:int(ratio * data.shape[0])]
vali_data = data[int(ratio * data.shape[0]):int((ratio + (1 - ratio) / 2) * data.shape[0])]
test_data = data[int((ratio + ( 1 - ratio ) / 2) * data.shape[0]):]
 
## Extract number of users and items
NUM_USERS = len(users)
NUM_ITEMS = len(items)

train_dataset = tf.data.Dataset.from_tensor_slices(train_data).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data).batch(batch_size)
print("===== Preprocess the data has been finished =====")

model = PMF(NUM_USERS, NUM_ITEMS, emb_dim)

print("===== Model Instantiated =====")

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=momentum)
# Instantiate a loss function.
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)

def train(train_dataset, len_dataset):
    
    epoch_loss = 0.0
    
    for step, elem in enumerate(train_dataset):
        row = elem[:, 0] ## users as a row
        col = elem[:, 1] ## items as a column
        val = elem[:, 2] ## normalized ratings as value
        
        with tf.GradientTape() as tape:
            
            ## Perlu diperiksa kembali
            row = tf.Variable(row, trainable=False)
            col = tf.Variable(col, trainable=False)
            val = tf.Variable(val, trainable=False)
            
            ## run the forward pass
            logits = model(row, col, training=True)
            
            ## compute the loss value
            loss_value = loss_fn(val, logits)
            
        ## using gradien tape to automatically retrieves
        ## the gradients of the trainable variables with respect to loss
        grads = tape.gradient(loss_value, model.trainable_weights)
        
        ## run one step of gradient descent by updating
        ## the value of variable loss
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        epoch_loss += loss_value
        
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
    
    epoch_loss /= len_dataset
    return float(epoch_loss)
    
print("===== Training Model =====")
## this list is used for visualization
train_loss_list = []
train_rmse_list = []
vali_rmse_list = []
len_train_data = len(train_data)

last_vali_rmse = None

for epoch in range(epoches):
    ## train epoch losses
    print("\nStart of epoch %d" % (epoch,))
    
    train_epoch_loss = train(train_dataset, len_train_data)
    
    ## test 
    train_loss_list.append(train_epoch_loss)
    
    ## creating index for predicting
    vali_row = tf.Variable(tf.convert_to_tensor(vali_data[:, 0]), trainable=False)
    vali_col = tf.Variable(tf.convert_to_tensor(vali_data[:, 1]), trainable=False)
    
    ## predicting the value
    vali_preds = model(vali_row, vali_col, training=False)
    
    ## calculating rmse
    train_rmse = np.sqrt(train_epoch_loss)
    vali_rmse = np.sqrt(np.mean(np.square(vali_preds-vali_data[:, 2])))
    
    train_rmse_list.append(train_rmse)
    vali_rmse_list.append(vali_rmse)
    
    print('Training epoch:{: d}, training rmse:{: .6f}, vali rmse:{:.6f}'.format(epoch+1, train_rmse, vali_rmse))
    
    if last_vali_rmse and last_vali_rmse < vali_rmse:
        break
    else:
        last_vali_rmse = vali_rmse

print("===== Testing Model =====")
test_row = tf.Variable(tf.convert_to_tensor(test_data[:, 0]), trainable=False)
test_col = tf.Variable(tf.convert_to_tensor(test_data[:, 1]), trainable=False)

preds = model(test_row, test_col, training=False)

test_rmse = np.sqrt(np.mean(np.square(preds-test_data[:, 2])))
print('Test rmse: {:f}'.format(test_rmse))

===== TRAINING PMF =====
===== Dataset has been loaded =====
===== Preprocess the data has been finished =====
===== Model Instantiated =====
===== Training Model =====

Start of epoch 0
Training loss (for one batch) at step 0: 0.4496
Training loss (for one batch) at step 200: 0.3219
Training loss (for one batch) at step 400: 0.3509
Training loss (for one batch) at step 600: 0.3678
Training loss (for one batch) at step 800: 0.3799
Training loss (for one batch) at step 1000: 0.3626
Training loss (for one batch) at step 1200: 0.4037
Training epoch: 1, training rmse: 0.077033, vali rmse:0.607329

Start of epoch 1
Training loss (for one batch) at step 0: 0.3879
Training loss (for one batch) at step 200: 0.3043
Training loss (for one batch) at step 400: 0.2921
Training loss (for one batch) at step 600: 0.3101
Training loss (for one batch) at step 800: 0.3336
Training loss (for one batch) at step 1000: 0.3172
Training loss (for one batch) at step 1200: 0.3275
Training epoch: 2, training rmse

In [ ]:
import matplotlib.pyplot as plt

# Create plots
plt.figure(1)
plt.plot(range(1, len(train_rmse_list)+1), train_rmse_list, color='r', label='train rmse')
plt.plot(range(1, len(vali_rmse_list)+1), vali_rmse_list, color='b', label='test rmse')
plt.legend()
plt.annotate(r'train=%f' % (train_rmse_list[-1]), xy=(len(train_rmse_list), train_rmse_list[-1]),
             xycoords='data', xytext=(-30, 30), textcoords='offset points', fontsize=10,
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3, rad=.2'))
plt.annotate(r'vali=%f' % (vali_rmse_list[-1]), xy=(len(vali_rmse_list), vali_rmse_list[-1]),
             xycoords='data', xytext=(-30, 30), textcoords='offset points', fontsize=10,
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3, rad=.2'))
plt.xlim([1, len(train_rmse_list)+10])
plt.xlabel('iterations')
plt.ylabel('RMSE')
plt.title('RMSE Curve in Training Process')
plt.show()

In [ ]:
inputs = tf.keras.Input(shape=(784,), name="digits")
x1 = tf.keras.layers.Dense(64, activation="relu")(inputs)
x2 = tf.keras.layers.Dense(64, activation="relu")(x1)
outputs = tf.keras.layers.Dense(10, name="predictions")(x2)
model = tf.keras.Model(inputs=inputs, outputs=outputs)